# 📦 Phase 1 — Data Collection
### BNPL Under the Microscope: Klarna, Affirm & the Future of Buy Now Pay Later

**Objective:** Pull all raw data needed for the project from public sources:
- Stock price data: KLAR, AFRM, PYPL, SQ via `yfinance`
- S&P 500 benchmark for relative performance
- BNPL market size & consumer debt risk data (manually curated from CFPB, NY Fed, Morgan Stanley)
- Klarna fundamentals (revenue, GMV, net income) from SEC filings

**Data Sources:**
- Yahoo Finance via `yfinance`
- CFPB BNPL Report Jan 2025 + Dec 2025 (public PDFs)
- NY Fed Consumer Credit Panel Q1 2025
- Morgan Stanley AlphaWise Survey 2025
- Klarna SEC F-1 filing & Q3 2025 earnings release
- Crunchbase / CB Insights funding data

In [1]:
# ── Install dependencies (run once) ──────────────────────────────────────────
!pip install yfinance pandas numpy matplotlib seaborn openpyxl

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

RAW_DIR = '../data/raw'
os.makedirs(RAW_DIR, exist_ok=True)
print('✅ Directories ready.')

## 1.1 — Stock Price Data

We pull daily adjusted closing prices for:
- **KLAR** — Klarna (IPO: Sep 10, 2025)
- **AFRM** — Affirm (Klarna's closest public competitor)
- **PYPL** — PayPal (BNPL via Pay Later)
- **SQ** — Block (owns Afterpay)
- **^GSPC** — S&P 500 (benchmark)

Note: KLAR only has data from Sep 10, 2025 onward. We normalise all prices to 100 at Klarna's IPO date for fair comparison.

In [ ]:
# ── Pull stock data ───────────────────────────────────────────────────────────
TICKERS = ['KLAR', 'AFRM', 'PYPL', 'SQ', '^GSPC']

# Klarna IPO date — we start all comparisons here
IPO_DATE = '2025-09-10'
END_DATE = '2026-02-12'

print(f'Pulling data from {IPO_DATE} to {END_DATE}...')
raw = yf.download(TICKERS, start=IPO_DATE, end=END_DATE, auto_adjust=True)

# Extract adjusted close prices
prices = raw['Close'].copy()
prices.columns.name = None

print(f'\n✅ Shape: {prices.shape}')
print(f'Date range: {prices.index[0].date()} → {prices.index[-1].date()}')
prices.tail()

In [ ]:
# ── Normalise prices to 100 at IPO date ──────────────────────────────────────
# This lets us compare performance on a level playing field
prices_norm = prices.div(prices.iloc[0]) * 100

print('Normalised prices (base = 100 on Sep 10, 2025):')
print(prices_norm.tail())

# Save
prices.to_csv(f'{RAW_DIR}/stock_prices_raw.csv')
prices_norm.to_csv(f'{RAW_DIR}/stock_prices_normalised.csv')
print('\n✅ Saved: stock_prices_raw.csv & stock_prices_normalised.csv')

In [ ]:
# ── Pull longer AFRM/PYPL/SQ history for revenue multiple analysis ────────────
COMP_TICKERS = ['AFRM', 'PYPL', 'SQ']
comp_hist = yf.download(COMP_TICKERS, start='2021-01-01', end='2026-02-12', auto_adjust=True)['Close']
comp_hist.columns.name = None
comp_hist.to_csv(f'{RAW_DIR}/competitor_prices_history.csv')
print(f'✅ Competitor history pulled: {comp_hist.shape}')

## 1.2 — Klarna Fundamentals (from F-1 / Earnings Releases)

These figures come directly from:
- Klarna Group plc F-1 filing (SEC, August 2025)
- Klarna Q3 2025 Earnings Press Release (November 18, 2025)
- CNBC, Bloomberg reporting on earnings

We build this as a structured DataFrame for analysis.

In [ ]:
# ── Klarna Annual Fundamentals 2019–2025E ─────────────────────────────────────
klarna_annual = pd.DataFrame({
    'year': [2019, 2020, 2021, 2022, 2023, 2024, 2025],
    'revenue_m': [602, 845, 1204, 1850, 2277, 2810, 3500],        # USD millions (2025 = estimate)
    'net_income_m': [-93, -153, -688, -1047, -241, 55, -200],     # Net income/loss (2025 = estimate)
    'gmv_b': [35, 53, 80, 100, 90, 105, 130],                     # Gross Merchandise Volume, USD billions
    'active_users_m': [70, 87, 90, 97, 100, 108, 114],            # Active consumers, millions
    'active_merchants_k': [190, 250, 400, 500, 550, 616, 850],    # Merchants, thousands
    'valuation_b': [5.5, 10.6, 45.6, 6.7, 9.0, 14.0, 15.0],      # Valuation at key funding/IPO events
    'headcount': [3500, 4000, 6500, 7000, 5200, 4300, 3800],      # Employees (reflects 40% AI cuts)
})

# Derived metrics
klarna_annual['revenue_growth_pct'] = klarna_annual['revenue_m'].pct_change() * 100
klarna_annual['arpu'] = (klarna_annual['revenue_m'] / klarna_annual['active_users_m']).round(2)  # Revenue per user
klarna_annual['take_rate_pct'] = (klarna_annual['revenue_m'] / (klarna_annual['gmv_b'] * 1000) * 100).round(3)  # Revenue / GMV

klarna_annual.to_csv(f'{RAW_DIR}/klarna_annual_fundamentals.csv', index=False)
print('✅ Klarna annual fundamentals saved')
klarna_annual

In [ ]:
# ── Klarna Quarterly Data (Q1 2024 – Q3 2025) ────────────────────────────────
klarna_quarterly = pd.DataFrame({
    'quarter': ['Q1 2024', 'Q2 2024', 'Q3 2024', 'Q4 2024', 'Q1 2025', 'Q2 2025', 'Q3 2025'],
    'revenue_m': [614, 706, 706, 784, 701, 823, 903],
    'net_income_m': [8, 12, 12, 30, -60, -57, -95],
    'gmv_b': [22.0, 24.5, 26.2, 31.0, 27.0, 29.5, 32.7],
    'active_users_m': [100, 102, 105, 108, 109, 111, 114],
    'merchants_k': [500, 550, 616, 680, 720, 800, 850],
})

klarna_quarterly['yoy_rev_growth'] = [None, None, None, None,
    (701/614-1)*100, (823/706-1)*100, (903/706-1)*100]
klarna_quarterly['yoy_rev_growth'] = klarna_quarterly['yoy_rev_growth'].round(1)

klarna_quarterly.to_csv(f'{RAW_DIR}/klarna_quarterly.csv', index=False)
print('✅ Klarna quarterly data saved')
klarna_quarterly

## 1.3 — BNPL Consumer Risk Data

Sourced from:
- CFPB BNPL Market Report, December 2025
- CFPB Consumer Use of BNPL Report, January 2025  
- Morgan Stanley AlphaWise Survey, April 2025
- LendingTree Consumer Survey, 2025
- Richmond Fed Economic Brief EB-25-03
- chargeflow.io BNPL Market Statistics aggregation

In [ ]:
# ── BNPL Late Payment Rates by Demographic ───────────────────────────────────
late_payments = pd.DataFrame({
    'demographic': ['Gen Z (18–26)', 'Millennials (27–42)', 'Gen X (43–58)', 'Boomers (59+)'],
    'late_payment_pct_2024': [44, 34, 8, 4],
    'late_payment_pct_2025': [51, 38, 10, 5],
    'overspent_pct_2025': [24, 35, 12, 6],
})
late_payments['yoy_change_pp'] = late_payments['late_payment_pct_2025'] - late_payments['late_payment_pct_2024']

late_payments.to_csv(f'{RAW_DIR}/bnpl_late_payments_demographics.csv', index=False)
print('✅ BNPL demographic data saved')
late_payments

In [ ]:
# ── Delinquency Comparison Across Credit Types ───────────────────────────────
delinquency = pd.DataFrame({
    'credit_type': [
        'BNPL (official default rate)',
        'BNPL (self-reported late payments)',
        'Overall consumer debt',
        'Credit cards',
        'Auto loans',
        'Student loans (90+ day)',
        'Klarna net charge-off rate',
    ],
    'delinquency_pct': [1.83, 41.0, 3.5, 8.8, 4.2, 7.7, 0.54],
    'source': [
        'CFPB Dec 2025',
        'LendingTree Survey 2025',
        'NY Fed Q1 2025',
        'NY Fed Q1 2025',
        'NY Fed Q1 2025',
        'NY Fed Q1 2025',
        'Klarna Q2 2025 Earnings',
    ]
})

delinquency.to_csv(f'{RAW_DIR}/delinquency_comparison.csv', index=False)
print('✅ Delinquency comparison saved')
delinquency

In [ ]:
# ── BNPL Market Size Projections ─────────────────────────────────────────────
market_size = pd.DataFrame({
    'year': [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028],
    'global_gmv_b': [35, 90, 186, 310, 420, 492, 560, 625, 695, 770],     # USD billions
    'us_market_b': [6, 20, 38, 60, 82, 103, 116.7, 130, 145, 163.8],
    'global_users_m': [40, 90, 180, 270, 360, 500, 650, 750, 850, 900],   # millions
    'is_projected': [False]*6 + [True]*4,
})

market_size.to_csv(f'{RAW_DIR}/bnpl_market_size.csv', index=False)
print('✅ BNPL market size data saved')
market_size

In [ ]:
# ── Competitor Fundamentals Snapshot ─────────────────────────────────────────
competitors = pd.DataFrame({
    'company': ['Klarna', 'Affirm', 'PayPal (Pay Later)', 'Block / Afterpay'],
    'ticker': ['KLAR', 'AFRM', 'PYPL', 'SQ'],
    'active_users_m': [114, 21, 400, 20],
    'revenue_2025_m': [3500, 2650, 31000, 22000],      # PayPal/Block are full company rev
    'revenue_growth_pct': [26, 36, 5, 8],
    'ipo_year': [2025, 2020, 2002, 2015],
    'market_cap_b_feb2026': [7.82, 15.0, 72.0, 38.0],
    'profitable': [False, False, True, False],
    'stock_return_since_klar_ipo': [-51, 12, -18, -8],  # % from Sep 10 2025
})

# Price / Revenue multiple (rough)
competitors['ps_ratio'] = (competitors['market_cap_b_feb2026'] * 1000 / competitors['revenue_2025_m']).round(2)

competitors.to_csv(f'{RAW_DIR}/competitor_snapshot.csv', index=False)
print('✅ Competitor snapshot saved')
competitors

In [ ]:
# ── Klarna Valuation History ──────────────────────────────────────────────────
valuation_history = pd.DataFrame({
    'date': ['2019-02', '2020-09', '2021-06', '2022-07',
             '2023-07', '2024-06', '2025-09-09', '2025-09-10', '2026-02-11'],
    'event': [
        'Series D', 'Series E', 'Series G (Peak)', 'Down-round',
        'Secondary trades', 'Pre-IPO round', 'IPO Pricing', 'IPO Day 1 Close', 'Current (Feb 2026)'
    ],
    'valuation_b': [5.5, 10.6, 45.6, 6.7, 9.0, 14.0, 15.0, 17.0, 7.82],
    'share_price_equiv': [None, None, None, None, None, None, 40.0, 45.82, 19.75],
    'type': ['private']*6 + ['public']*3
})

valuation_history.to_csv(f'{RAW_DIR}/klarna_valuation_history.csv', index=False)
print('✅ Valuation history saved')
valuation_history

In [ ]:
# ── Final Summary of all saved files ─────────────────────────────────────────
print('\n📁 Files saved to data/raw/:')
for f in sorted(os.listdir(RAW_DIR)):
    size = os.path.getsize(f'{RAW_DIR}/{f}')
    print(f'   {f:45s}  {size:>8,} bytes')

print('\n✅ Phase 1 complete. Proceed to 02_eda_klarna_stock.ipynb')